In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pathlib
import time
import os
import random

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import re
import pandas as pd

from tqdm import tqdm
import numpy as np
import urllib.request

In [33]:
class NetflixOfficialCrwl:
    def __init__(self,path):
        self.chrome_options = Options()
        self.chrome_options.add_experimental_option("detach", True)
        self.chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
        self.service = Service(executable_path=ChromeDriverManager().install())

        self.chrome_options.add_argument('--no-sandbox')
        self.chrome_options.add_argument('--disable-dev-shm-usage')
        self.chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")
        # self.chrome_options.add_argument(f'user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36')

        self.path = pathlib.Path(f'{path}')
        self.netflix_csv = pd.read_csv(self.path)

    def netflix_official_crwl(self,start_number=0):
        n_title = self.netflix_csv['nfid'][start_number:]
        driver = webdriver.Chrome(service=self.service,options=self.chrome_options)
        driver.implicitly_wait(10)
        result = pd.DataFrame()
        not_find = []
        img_not_find = []
        for n_t in tqdm(n_title):
            url = 'https://www.netflix.com/kr/title/'
            url += str(n_t)
            driver.get(url)
            time.sleep(round(0.5+random.random(),2))
            try:
                title = driver.find_element(By.CSS_SELECTOR,'h1.title-title').text
                title = {'제목':title}
                try:
                    synopsis = driver.find_element(By.CSS_SELECTOR,'div.title-info-synopsis').text
                    synopsis = {'시놉시스':synopsis}
                except:
                    synopsis = {'시놉시스':np.nan}

                genres = driver.find_elements(By.CSS_SELECTOR,'span.more-details-item.item-genres')
                genres_str= ''
                for g in genres:
                    genres_str+=g.text
                genres_str = {'장르모음':genres_str}

                people = driver.find_elements(By.CSS_SELECTOR,'span.more-details-item.item-cast')
                people_str= ''
                for p in people:
                    people_str+=p.text+','
                people_str = {'출연진':people_str}
                try:
                    feature = driver.find_elements(By.CSS_SELECTOR,'span.more-details-item.item-mood-tag')
                    feature_str = ''
                    for f in feature:
                        feature_str+=f.text
                    feature_str = {'특징':feature_str}     
                except:
                    feature_str = {'특징':np.nan}     


                try:
                    img=driver.find_element(By.CSS_SELECTOR,'div.hero-image.hero-image-desktop')
                    img = img.get_attribute('style')
                    img_src = {'img_src':re.search('"(.+?)"',img).group(1)}
                    urllib.request.urlretrieve(img_src['img_src'],f'./img/{n_t}.jpg')
                except:
                    print(f'img not find {n_t}')
                    img_not_find.append(n_t)
                    img_src = {'img_src':np.nan}
                metadata = driver.find_element(By.CSS_SELECTOR,'div.title-info-metadata-wrapper').text.split('|')
                if len(metadata)==3: metadata.insert(2,'0')
                    
                
                api_netflix = self.netflix_csv[self.netflix_csv['nfid']==n_t][['nfid','imdbid']].values[0]
                n_i_id = {'nfid':api_netflix[0],'imdbid':api_netflix[1]}

                df = pd.concat([pd.DataFrame([title]),pd.DataFrame([genres_str]),pd.DataFrame([synopsis]),pd.DataFrame([feature_str]),
                                pd.DataFrame([people_str]),pd.DataFrame([metadata],columns=['개봉일','등급','상영시간','기본장르']),pd.DataFrame([n_i_id])]
                                ,axis=1)
                result = pd.concat([result,df])
                # result.to_csv('test.csv',encoding='utf-8')
            except:
                # result.to_csv(f'n_{start_number}.csv',encoding='utf-8')
                print(n_t)
                not_find.append(n_t)
                continue
        result.to_csv('netflix_official_.csv',encoding='utf-8')
        pd.DataFrame(not_find).to_csv('not.find.csv',encoding='utf-8')
                
    

In [34]:
crwl = NetflixOfficialCrwl(path='../../../data/netflix_plus.csv')
crwl.netflix_official_crwl()

  4%|▍         | 78/1742 [08:37<3:35:42,  7.78s/it]

80222720


  5%|▍         | 83/1742 [09:15<3:28:07,  7.53s/it]

81046957


 55%|█████▌    | 963/1742 [1:46:27<1:18:36,  6.05s/it]

80217151


100%|██████████| 1742/1742 [3:18:04<00:00,  6.82s/it]  
